In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook')

In [2]:
img_metadata = pd.read_csv("../dataset/Data_Entry_2017.csv")
unusable_images = pd.read_csv("../dataset/cxr14_bad_labels.csv").drop(["Index"], axis=1)

In [3]:
finding_labels = set()
for labels in pd.unique(img_metadata["Finding Labels"]):
    for label in labels.split("|"):
        finding_labels.add(label)

print("All labels:")
for i, label in enumerate(finding_labels):
    print(f"{i + 1}. {label}")
print(f"Total labels: {len(finding_labels)}")

All labels:
1. Cardiomegaly
2. Edema
3. Emphysema
4. No Finding
5. Fibrosis
6. Mass
7. Atelectasis
8. Pleural_Thickening
9. Hernia
10. Infiltration
11. Pneumonia
12. Pneumothorax
13. Nodule
14. Effusion
15. Consolidation
Total labels: 15


In [4]:
img_metadata.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'Image Width',
       'Image Height', 'Spacing X', 'Spacing Y'],
      dtype='object')

In [5]:
img_metadata.rename(columns = {"Image Index": "img_filename", "Patient ID": "pt_id", "Patient Age": "pt_age", "Patient Gender": "pt_sex", "View Position": "view_position", "Image Width": "img_width", "Image Height": "img_height", "Spacing X": "x_spacing", "Spacing Y": "y_spacing"}, inplace=True)
img_metadata.columns

Index(['img_filename', 'Finding Labels', 'Follow-up #', 'pt_id', 'pt_age',
       'pt_sex', 'view_position', 'img_width', 'img_height', 'x_spacing',
       'y_spacing'],
      dtype='object')

In [6]:
def has_label(label, row):
    return 1 if label in row["Finding Labels"].split("|") else 0

for label in finding_labels:
    img_metadata[label] = img_metadata.apply(lambda row: has_label(label, row), axis=1)

img_metadata = img_metadata.drop(["Finding Labels", "Follow-up #"], axis=1)

In [7]:
unusable_images.head()

,File label,Inverted,Not frontal,Rotated
0,00000583_024.png,1.0,0.0,0.0
1,00002180_000.png,1.0,0.0,0.0
2,00002300_026.png,1.0,0.0,0.0
3,00002371_015.png,1.0,0.0,0.0
4,00006209_001.png,1.0,0.0,0.0


In [8]:
to_drop = []
unusable_list = unusable_images["File label"].tolist()
for index, row in img_metadata.iterrows():
    if row["img_filename"] in unusable_list:
        to_drop.append(index)
print(f"Total images to drop: {len(pd.unique(unusable_images['File label']))}")
print(f"Total indexes found: {len(to_drop)}")


Total images to drop: 257
Total indexes found: 257


In [9]:
usable_imgs = img_metadata.drop(to_drop, axis=0)
print(f"Total images before drop: {len(img_metadata)}")
print(f"Images usable: {len(usable_imgs)}")
print(f"Total dropped: {len(img_metadata) - len(usable_imgs)}")

Total images before drop: 112120
Images usable: 111863
Total dropped: 257


In [10]:
usable_imgs.to_csv("../dataset/usable_img_metadata.csv", index=False)